# Training fastText

In [0]:
from google.colab import drive

drive.mount("/drive/")

In [0]:
root = "/drive/My Drive"
path = "/Shared/FYP/Data/"
filename = "roman_processed.txt"

## Loading the corpus

In [0]:
import time
import logging
from gensim.models.word2vec import LineSentence

logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

lines = LineSentence(root + path + filename)

## Training a fastText model

### Usage
FastText(sentences=None, corpus_file=None, sg=0, hs=0, size=100, alpha=0.025, window=5, min_count=5, max_vocab_size=None, word_ngrams=1, sample=0.001, seed=1, workers=3, min_alpha=0.0001, negative=5, ns_exponent=0.75, cbow_mean=1, hashfxn=<built-in function hash>, iter=5, null_word=0, min_n=3, max_n=6, sorted_vocab=1, bucket=2000000, trim_rule=None, batch_words=10000, callbacks=(), compatible_hash=True)


**Parameters:**
1. **sentences** (iterable of list of str, optional) – Can be simply a list of lists of tokens, but for larger corpora, consider an iterable that streams the sentences directly from disk/network. See BrownCorpus, Text8Corpus or LineSentence in word2vec module for such examples. If you don’t supply sentences, the model is left uninitialized – use if you plan to initialize it in some other way.
2. **corpus_file** (str, optional) – Path to a corpus file in LineSentence format. You may use this argument instead of sentences to get performance boost. Only one of sentences or corpus_file arguments need to be passed (or none of them, in that case, the model is left uninitialized).
3. **min_count** (int, optional) – The model ignores all words with total frequency lower than this.
4. **size** (int, optional) – Dimensionality of the word vectors.
5. **window** (int, optional) – The maximum distance between the current and predicted word within a sentence.
6. **workers** (int, optional) – Use these many worker threads to train the model (=faster training with multicore machines).
7. **alpha** (float, optional) – The initial learning rate.
8. **min_alpha** (float, optional) – Learning rate will linearly drop to min_alpha as training progresses.
9. **sg** ({1, 0}, optional) – Training algorithm: skip-gram if sg=1, otherwise CBOW.
10. **hs** ({1,0}, optional) – If 1, hierarchical softmax will be used for model training. If set to 0, and negative is non-zero, negative sampling will be used.
11. **seed** (int, optional) – Seed for the random number generator. Initial vectors for each word are seeded with a hash of the concatenation of word + str(seed). Note that for a fully deterministically-reproducible run, you must also limit the model to a single worker thread (workers=1), to eliminate ordering jitter from OS thread scheduling. (In Python 3, reproducibility between interpreter launches also requires use of the PYTHONHASHSEED environment variable to control hash randomization).
12. **max_vocab_size** (int, optional) – Limits the RAM during vocabulary building; if there are more unique words than this, then prune the infrequent ones. Every 10 million word types need about 1GB of RAM. Set to None for no limit.
13. **sample** (float, optional) – The threshold for configuring which higher-frequency words are randomly downsampled, useful range is (0, 1e-5).
14. **negative** (int, optional) – If > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). If set to 0, no negative sampling is used.
15. **ns_exponent** (float, optional) – The exponent used to shape the negative sampling distribution. A value of 1.0 samples exactly in proportion to the frequencies, 0.0 samples all words equally, while a negative value samples low-frequency words more than high-frequency words. The popular default value of 0.75 was chosen by the original Word2Vec paper. More recently, in https://arxiv.org/abs/1804.04212, Caselles-Dupré, Lesaint, & Royo-Letelier suggest that other values may perform better for recommendation applications.
16. **cbow_mean** ({1,0}, optional) – If 0, use the sum of the context word vectors. If 1, use the mean, only applies when cbow is used.
17. **hashfxn** (function, optional) – Hash function to use to randomly initialize weights, for increased training reproducibility.
18. **iter** (int, optional) – Number of iterations (epochs) over the corpus.
19. **trim_rule** (function, optional) –

    Vocabulary trimming rule, specifies whether certain words should remain in the vocabulary, be trimmed away, or handled using the default (discard if word count < min_count). Can be None (min_count will be used, look to keep_vocab_item()), or a callable that accepts parameters (word, count, min_count) and returns either gensim.utils.RULE_DISCARD, gensim.utils.RULE_KEEP or gensim.utils.RULE_DEFAULT. The rule, if given, is only used to prune vocabulary during build_vocab() and is not stored as part of themodel.

    The input parameters are of the following types:
            word (str) - the word we are examining
            count (int) - the word’s frequency count in the corpus
            min_count (int) - the minimum count threshold.

20. **sorted_vocab** ({1,0}, optional) – If 1, sort the vocabulary by descending frequency before assigning word indices.
21. **batch_words** (int, optional) – Target size (in words) for batches of examples passed to worker threads (and thus cython routines).(Larger batches will be passed if individual texts are longer than 10000 words, but the standard cython code truncates to that maximum.)
22. **min_n** (int, optional) – Minimum length of char n-grams to be used for training word representations.
23. **max_n** (int, optional) – Max length of char ngrams to be used for training word representations. Set max_n to be lesser than min_n to avoid char ngrams being used.
24. **word_ngrams** ({1,0}, optional) – If 1, uses enriches word vectors with subword(n-grams) information. If 0, this is equivalent to Word2Vec.
25. **bucket** (int, optional) – Character ngrams are hashed into a fixed number of buckets, in order to limit the memory usage of the model. This option specifies the number of buckets used by the model.
26. **callbacks** – List of callbacks that need to be executed/run at specific stages during training.
27. **compatible_hash** (bool, optional) – By default, newer versions of Gensim’s FastText use a hash function that is 100% compatible with Facebook’s FastText. Older versions were not 100% compatible due to a bug. To use the older, incompatible hash function, set this to False.

In [0]:
import multiprocessing
from gensim.models import FastText

size = 500                               # Vector dimension
window = 5                               # Context window size
min_count = 1                            # Cut off frequency
sg = 0                                   # 0 for CBOW, 1 for Skip-gram
workers = multiprocessing.cpu_count()    # Number of cores to use

model = FastText(size=size, window=window, min_count=min_count, sg=sg, workers=workers)

In [0]:
def get_model_name():
    model_name = 'fasttext'
    
    if 'roman' in filename:
        model_name += '_roman'
    else:
        model_name += '_urdu'
        
    if sg == 0:
        model_name += '_cbow'
    else:
        model_name += '_sg'
        
    model_name += '_' + str(size)
    return model_name

In [0]:
t = time.time()

model.build_vocab(sentences=lines, progress_per=1000000)

print('Time to build vocab: {} mins'.format(round((time.time() - t) / 60, 2)))

In [0]:
model_name = get_model_name()
print("About to train {}".format(model_name))

t = time.time()

model.train(sentences=lines, total_examples=model.corpus_count, epochs=5, report_delay=10)

print('Time to train the model: {} mins'.format(round((time.time() - t) / 60, 2)))

## Saving the model

In [0]:
model.save(model_name)